__This programme is the implementation of a machine-learning based robust optimization approach introduced in http://www.optimization-online.org/DB_FILE/2017/06/6057.pdf__

__Generating data samples using multi variate normal distribution__

In [ ]:
from scipy.linalg import sqrtm
from scipy.linalg import eig
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal
n=500 
sampl = np.random.uniform(low=0.1, high=0.7, size=(9,9))
mean = [-20736.00,17.28,17.28,17.28,17.28,17.28,17.28,17.28,17.28]
cov = np.dot(sampl,sampl.T) 
Samples = np.random.multivariate_normal(mean, cov, n).T               


C=np.zeros((n,9))
for i in range(9):
    C[:,i]=Samples[i]

__Start of steps for data driven roust optimization approach__

In [2]:
#A function that takes n vectors (rows of A are the samples) and returns the mean value of each dimension
def _meanvalue(A):
    n=len(A[:,0]) #n is the number of rows that is number of samples
    m=len(A[0,:])  # is the number of dimension
    meanvalue=[]
    for i in range(m):
        t=sum(A[:,i])
        t=1/n*t
        meanvalue.append(t)
    return meanvalue


In [3]:
def matmulti(A,B):   #A function that multiplies entries of two matrix
    n=len(A)
    K=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            K[i][j]=A[i]*B[j]
    return K


In [4]:
#This block computes the whitening matrix (whiten = Co^-1/2)


def _Whiten(A):
    meanvalue=_meanvalue(A)
    n=len(A[:,0])
    m=len(A[0,:])
    Co=np.zeros((m,m))
    for i in range(n):
        B= A[i,:]-np.array(meanvalue)
        K=matmulti(B,B)
        Co=Co+KK
    Co=1/(n-1)*Co
    Sigma=sqrtm(Co)
    Whiten=np.linalg.pinv(Sigma)
    return Whiten


In [5]:
def transformer(A):   #سطرهای ماتریس ورودی نمونه های ما هستند
    Whiten=_Whiten(A)  #خروجی این تابع دیتاهایی با کوواریانس قطری است
    transformed=np.dot(Whiten,A.T)
    return transformed    #ستونهای ماتریس خروجی، نمونه های جدیدمون هستند

In [6]:
def kernelparam(A):# this function computes kernel parametres l_k. l_k is the distance between max and min values of 
                                              #kth variable
    transformed=transformer(A)
    Lower=[]
    for i in range(len(transformed[:,0])):
        Lower.append(min(transformed[i,:]))
    Upper=[]
    for i in range(len(transformed[:,0])):
        Upper.append(max(transformed[i,:])) 
    upperminuslower=[]
    for i,j in zip(Upper,Lower):
        upperminuslower.append(i-j)
    L=np.array(upperminuslower)+1   #Here we want to add a positive number to the difference between max and min to make l_k 
                                     #greater than the difference
    return L

In [7]:
# Step 3   ....    Formin the matrix K

# K(u,v)=sum(L_k)-||whitening(u-v)||_1

def kernelmatrix(A): 
    Whiten=_Whiten(A)
    n=len(A[:,0])
    m=len(A[0,:])
    K=np.zeros((n,n))
    L=kernelparam(A)
    l=sum(L)
    transformedsample=np.dot(Whiten,A.T)
    for i in range(n):
        for j in range(n):
            K[i,j]=l-sum(   [abs((transformedsample[:,i]-transformedsample[:,j])[r]) for r in range(m)])
    return K

__Solving the quadratic optimization problem to find the support vectors__

In [8]:
from pyomo.environ import *
opt=SolverFactory('ipopt')
quad=ConcreteModel()

In [9]:
n=len(C[:,0])
nu=0.7  #Conservatism level
_I=[i for i in range(n)]
quad.I=Set(initialize=_I)
quad.alpha=Var(quad.I , within=NonNegativeReals)
def _c1(quad,i):
    return  quad.alpha[i]<=1/(n*nu)
quad.c1=Constraint(quad.I, rule=_c1)
def _c2(quad):
    return sum(quad.alpha[i] for i in quad.I)==1
quad.c2=Constraint(rule=_c2)
import numpy as np
#K=kernelmatrix(A)
K=kernelmatrix(C)

def _objective(quad):
    return sum(quad.alpha[i]*quad.alpha[j]*K[i][j] for i in quad.I for j in quad.I) - sum(quad.alpha[i]*K[i][i] for i in quad.I)
quad.z=Objective(rule=_objective , sense=minimize)

In [10]:
results = opt.solve(quad)


In [11]:
results


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 251, 'Number of variables': 250, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.11.1\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 3.109177827835083}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [12]:
results.write(num=1)


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 251
  Number of variables: 250
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 3.109177827835083
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In [13]:
quad.display()


Model unknown

  Variables:
    alphaa : Size=250, Index=II
        Key : Lower : Value                  : Upper : Fixed : Stale : Domain
          0 :     0 :                    0.0 :  None : False : False : NonNegativeReals
          1 :     0 :   0.005714295187678057 :  None : False : False : NonNegativeReals
          2 :     0 :                    0.0 :  None : False : False : NonNegativeReals
          3 :     0 :  0.0057142952168792475 :  None : False : False : NonNegativeReals
          4 :     0 :                    0.0 :  None : False : False : NonNegativeReals
          5 :     0 :                    0.0 :  None : False : False : NonNegativeReals
          6 :     0 :   0.005714294712297731 :  None : False : False : NonNegativeReals
          7 :     0 :   0.005714294911374189 :  None : False : False : NonNegativeReals
          8 :     0 :    0.00571429533315465 :  None : False : False : NonNegativeReals
          9 :     0 :    0.00571429486206871 :  None : False : False :

        228 :     0 :  0.0057142843373893994 :  None : False : False : NonNegativeReals
        229 :     0 :   0.005714295528853218 :  None : False : False : NonNegativeReals
        230 :     0 :                    0.0 :  None : False : False : NonNegativeReals
        231 :     0 :   0.005714295459685318 :  None : False : False : NonNegativeReals
        232 :     0 :   0.005714295064857217 :  None : False : False : NonNegativeReals
        233 :     0 :   0.005714295486991039 :  None : False : False : NonNegativeReals
        234 :     0 :                    0.0 :  None : False : False : NonNegativeReals
        235 :     0 :   0.005714295569328096 :  None : False : False : NonNegativeReals
        236 :     0 : 4.1355108146049434e-09 :  None : False : False : NonNegativeReals
        237 :     0 :   0.005714294914303647 :  None : False : False : NonNegativeReals
        238 :     0 :  0.0057142954077378984 :  None : False : False : NonNegativeReals
        239 :     0 :    0.00571

In [14]:
print(value(quad.z))

-11.24274482045756


In [15]:
SV=[]  #support vectors
BSV=[]   #boundary support vectors
AlphaValues={}
for i in range(n):
    AlphaValues[i]=value(quad.alpha[i])
    if AlphaValues[i]>0:
        SV.append(i)
    if AlphaValues[i]<1/(n*nu) and AlphaValues[i]>0:
        BSV.append(i)
print(len(BSV))

10


In [ ]:
#Stage 5  
Q=_Whiten(C)
i=BSV[0]
teta=0
for i in SV:
    teta=teta+AlphaValues[i]*sum(abs(np.dot(Q,C[i,:]-C[i,:])))

print(len(SV))
BSV